##    digger the features
including:
- minimum/ 
- max / 
- ave / ave normalized
- start / start normalized
- end / end normalized
- difference between start to end / normalized
- difference between start to max / normalized
- difference between start to min / normalized
- std
- the number of different price
- up number / rate
- down number /rate
- length
- duration
- Bias: max, min, ave

In [2]:
import pandas as pd
import os
import numpy as np
# train data
summary_folder_path = '../../datasets/train_data/summary'
mining_folder_path = '../../datasets/train_data/data_mining'
result_folder_path = '../../datasets/rawdata'
games = { 'baseball':'/baseball/mlb',
         'iceball': '/iceball/nhl',
         'soccer_champion':'/soccer/champ_league',
         'scoccer_england':'/soccer/epl',
         'soccer_major':'/soccer/majorleague'}
# folder name is games, result name is games_results.csv

games_list = ['baseball','iceball']
soccer_list = ['soccer_champion','scoccer_england','soccer_major']

In [103]:
def diggers(df,home_name):
    # input is the frame, price_series_name
    # output is summary
    if len(df) < 1:
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    odds_series = df[home_name].tolist()
    
#     odds
    draw_odds = df.loc[:,'draw'].iloc[0],
    away_odds = df.loc[:,'away'].iloc[0]
    draw_odds = list(draw_odds)[0]
    # length
    length = len(odds_series)
    # duration
    duration = df.iloc[-1,0]
    # bias
    bias = abs(df[home_name] - df['home'])
    bias_max = bias.max()
    bias_min = bias.min()
    bias_ave = bias.mean()
    bias_std = bias.std()
    # maximum # minimum
    maximum = max(odds_series)
    minimum = min(odds_series)
    range_ = maximum - minimum
    if range_ == 0:
        return [maximum, maximum, maximum, 1., maximum, 1., maximum, 1., 0., 0., 0., 0., 0., 0., 0., 1., length, 
            0., 0., 0., 0., duration, bias_max, bias_min, bias_ave, bias_std, draw_odds, away_odds]
    #ave
    ave = np.mean(np.array([odds_series]), axis=1)[0]
    ave_normalized = (ave - minimum)/range_
    #start
    start = odds_series[-1]
    start_normalized = (start - minimum)/range_
    # end 
    end = odds_series[0]
    end_normalized = (end - minimum)/range_
    # difference
    start2end = end - start
    start2end_normalized = (start2end - minimum)/range_
    start2max = maximum - start
    start2max_normalized = (start2max - minimum)/range_
    start2min = minimum - start
    start2min_normalized = (start2min - minimum)/range_
    # std
    std = np.std(np.array([odds_series]), axis=1)[0]
    # number of price
    no_price = len(set(odds_series))
    # tendency
    up_num = 0
    down_num = 0
    for i in range(length-1):
        if odds_series[i] < odds_series[i+1]:
            up_num += 1
        elif odds_series[i] > odds_series[i+1]:
            down_num += 1
        else:
            pass
    up_rate = up_num/(up_num+down_num)
    down_rate = down_num/(up_num+down_num)

    
    return [maximum, minimum, ave, ave_normalized, 
            start, start_normalized, end, end_normalized, start2end,
            start2end_normalized, start2max, start2max_normalized, start2min, start2min_normalized, 
            std, no_price, length, 
            up_num, down_num, up_rate, down_rate, duration, 
            bias_max, bias_min, bias_ave, bias_std, draw_odds, away_odds]
    

In [105]:
unbiased = pd.DataFrame()
hourly_corrected = pd.DataFrame()
daily_corrected = pd.DataFrame()
global_corrected = pd.DataFrame()

features_name = ['maximum', 'minimum', 'ave', 'ave_normalized', 
            'start', 'start_normalized', 'end', 'end_normalized', 'start2end',
            'start2end_normalized', 'start2max', 'start2max_normalized', 'start2min', 'start2min_normalized', 
            'std', 'no_price', 'length', 
            'up_num', 'down_num', 'up_rate', 'down_rate', 'duration', 
            'bias_max', 'bias_min', 'bias_ave', 'bias_st', 'draw_odds','away_odds']

for game_name in soccer_list:
    print(game_name)
    results = pd.read_csv(result_folder_path+games[game_name]+'_results.csv').set_index('Unnamed: 0')
    index_list = [i[:-4] for i in os.listdir(summary_folder_path + games[game_name])]
    for index in index_list:
        series_df = pd.read_csv(summary_folder_path + games[game_name]+'/'+str(index)+'.csv')
#         if len(series_df) < 7:
#             continue
        # sec 3
        sec3 = series_df[series_df['duration']<= int(series_df.iloc[-1,0]/3)] # sec 3
        # sec 2
        sec2 = series_df[(series_df['duration']<= int(series_df.iloc[-1,0]/3*2)) \
                  & (series_df['duration']> int(series_df.iloc[-1,0]/3)) ] #sec 2
        # sec 1
        sec1 = series_df[series_df['duration']> int(series_df.iloc[-1,0]/3*2)] # sec 1        
        
        index = int(index)
        if results.loc[index,'home_score'] ==  results.loc[index,'away_score'] :
            win = {'draw':1,'home':0,'away':0}
        elif results.loc[index,'home_score'] >  results.loc[index,'away_score'] :
            win = {'draw':0,'home':1,'away':0}
        else:
            win = {'draw':0,'home':0,'away':1}
        
#         these four frame will be digger by different series
        series_name = 'home'
        features = diggers(series_df,series_name)
        feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
        features = diggers(sec1,series_name)
        feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
        features = diggers(sec2,series_name)
        feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
        features = diggers(sec3,series_name)
        feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))
        
        features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
        unbiased = unbiased.append(features, ignore_index=True)
        
        series_name = 'hourly_corrected'
        features = diggers(series_df,series_name)
        feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
        features = diggers(sec1,series_name)
        feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
        features = diggers(sec2,series_name)
        feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
        features = diggers(sec3,series_name)
        feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))
        
        features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
        hourly_corrected = hourly_corrected.append(features, ignore_index=True)
        
        
        series_name = 'daily_corrected'
        features = diggers(series_df,series_name)
        feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
        features = diggers(sec1,series_name)
        feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
        features = diggers(sec2,series_name)
        feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
        features = diggers(sec3,series_name)
        feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))
        
        features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
        daily_corrected = daily_corrected.append(features, ignore_index=True)
        
        
        series_name = 'global_corrected'
        features = diggers(series_df,series_name)
        feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
        features = diggers(sec1,series_name)
        feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
        features = diggers(sec2,series_name)
        feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
        features = diggers(sec3,series_name)
        feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))
        
        features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
        global_corrected = global_corrected.append(features, ignore_index=True)

unbiased.to_csv(mining_folder_path+'/soccer_unbiased.csv')
hourly_corrected.to_csv(mining_folder_path+'/soccer_hourly_corrected.csv')
daily_corrected.to_csv(mining_folder_path+'/soccer_daily_corrected.csv')
global_corrected.to_csv(mining_folder_path+'/soccer_global_corrected.csv')

soccer_champion
0.0
0.0
0.0
0.0
0.0051793511906694486
0.0
0.005183702004014105
0.0023487840957972157
0.003240424175481341
0.0
0.0037943948192118004
0.003833020936460039
0.005506941806852526
0.0
0.006021209400695194
0.00632920811801988
0.0
0.0
0.0
0.0
0.00819336929796486
0.005224075362392198
0.003539509446309003
0.0
0.0003284532629375472
0.0
0.0004192934100899039
0.0
0.0023784179342277952
0.0
0.003036215738404696
0.0
0.0
0.0
0.0
0.0
0.012117469759113897
0.0019358105349225967
0.008179535250632596
0.012891152733551197
0.006073428388464828
0.002844372083406111
0.0
0.00648061848301113
0.004191915220591443
0.0
0.0
0.0041609141498570885
0.0
0.0
0.0
0.0
0.00788485780937462
0.005823318640220949
0.006181375997258087
0.006863223751218735
0.005128062788090247
0.007325275467402826
0.0037552826600305853
0.004179515436861045
0.004077914345678984
0.003046286832820477
0.005226444058513403
0.0039653485427323595
0.0
0.0
nan
0.0
0.09583983374750311
0.0
nan
0.012206319150358101
0.019548253317156036
0.01289

In [99]:
def diggers(df,home_name):
    # input is the frame, price_series_name
    # output is summary
    if len(df) < 1:
        return [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    odds_series = df[home_name].tolist()
    
    
    # length
    length = len(odds_series)
    # duration
    duration = df.iloc[-1,0]
    # bias
    bias = abs(df[home_name] - df['home'])
    bias_max = bias.max()
    bias_min = bias.min()
    bias_ave = bias.mean()
    bias_std = bias.std()
    # maximum # minimum
    maximum = max(odds_series)
    minimum = min(odds_series)
    range_ = maximum - minimum
    if range_ == 0:
        return [maximum, maximum, maximum, 1, maximum, 1, maximum, 1, 0,0, 0, 0, 0, 0, 0, 1, length, 
            0, 0, 0, 0, duration, bias_max, bias_min, bias_ave, bias_std]
    #ave
    ave = np.mean(np.array([odds_series]), axis=1)[0]
    ave_normalized = (ave - minimum)/range_
    #start
    start = odds_series[-1]
    start_normalized = (start - minimum)/range_
    # end 
    end = odds_series[0]
    end_normalized = (end - minimum)/range_
    # difference
    start2end = end - start
    start2end_normalized = (start2end - minimum)/range_
    start2max = maximum - start
    start2max_normalized = (start2max - minimum)/range_
    start2min = minimum - start
    start2min_normalized = (start2min - minimum)/range_
    # std
    std = np.std(np.array([odds_series]), axis=1)[0]
    # number of price
    no_price = len(set(odds_series))
    # tendency
    up_num = 0
    down_num = 0
    for i in range(length-1):
        if odds_series[i] < odds_series[i+1]:
            up_num += 1
        elif odds_series[i] > odds_series[i+1]:
            down_num += 1
        else:
            pass
    up_rate = up_num/(up_num+down_num)
    down_rate = down_num/(up_num+down_num)

    
    return [maximum, minimum, ave, ave_normalized, 
            start, start_normalized, end, end_normalized, start2end,
            start2end_normalized, start2max, start2max_normalized, start2min, start2min_normalized, 
            std, no_price, length, 
            up_num, down_num, up_rate, down_rate, duration, 
            bias_max, bias_min, bias_ave, bias_std]

In [102]:
unbiased = pd.DataFrame()
hourly_corrected = pd.DataFrame()
daily_corrected = pd.DataFrame()
global_corrected = pd.DataFrame()

features_name = ['maximum', 'minimum', 'ave', 'ave_normalized', 
            'start', 'start_normalized', 'end', 'end_normalized', 'start2end',
            'start2end_normalized', 'start2max', 'start2max_normalized', 'start2min', 'start2min_normalized', 
            'std', 'no_price', 'length', 
            'up_num', 'down_num', 'up_rate', 'down_rate', 'duration', 
            'bias_max', 'bias_min', 'bias_ave', 'bias_st']

game_name = 'baseball'
results = pd.read_csv(result_folder_path+games[game_name]+'_results.csv').set_index('Unnamed: 0')
index_list = [i[:-4] for i in os.listdir(summary_folder_path + games[game_name])]
for index in index_list:
    series_df = pd.read_csv(summary_folder_path + games[game_name]+'/'+str(index)+'.csv')
#     if len(series_df) < 7:
#         continue
    # sec 3
    sec3 = series_df[series_df['duration']<= int(series_df.iloc[-1,0]/3)] # sec 3
    # sec 2
    sec2 = series_df[(series_df['duration']<= int(series_df.iloc[-1,0]/3*2)) \
              & (series_df['duration']> int(series_df.iloc[-1,0]/3)) ] #sec 2
    # sec 1
    sec1 = series_df[series_df['duration']> int(series_df.iloc[-1,0]/3*2)] # sec 1        

    index = int(index)
    if results.loc[index,'home_score'] >  results.loc[index,'away_score'] :
        win = {'home':1,'away':0}
    else:
        win = {'home':0,'away':1}

#         these four frame will be digger by different series
    series_name = 'home'
    features = diggers(series_df,series_name)
    feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
    features = diggers(sec1,series_name)
    feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
    features = diggers(sec2,series_name)
    feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
    features = diggers(sec3,series_name)
    feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))

    features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
    unbiased = unbiased.append(features, ignore_index=True)


    series_name = 'hourly_corrected'
    features = diggers(series_df,series_name)
    feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
    features = diggers(sec1,series_name)
    feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
    features = diggers(sec2,series_name)
    feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
    features = diggers(sec3,series_name)
    feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))

    features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
    hourly_corrected = hourly_corrected.append(features, ignore_index=True)


    series_name = 'daily_corrected'
    features = diggers(series_df,series_name)
    feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
    features = diggers(sec1,series_name)
    feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
    features = diggers(sec2,series_name)
    feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
    features = diggers(sec3,series_name)
    feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))

    features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
    daily_corrected = daily_corrected.append(features, ignore_index=True)


    series_name = 'global_corrected'
    features = diggers(series_df,series_name)
    feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
    features = diggers(sec1,series_name)
    feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
    features = diggers(sec2,series_name)
    feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
    features = diggers(sec3,series_name)
    feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))

    features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
    global_corrected = global_corrected.append(features, ignore_index=True)

unbiased.to_csv(mining_folder_path+'/baseball_unbiased.csv')
hourly_corrected.to_csv(mining_folder_path+'/baseball_hourly_corrected.csv')
daily_corrected.to_csv(mining_folder_path+'/baseball_daily_corrected.csv')
global_corrected.to_csv(mining_folder_path+'/baseball_global_corrected.csv')

In [101]:
unbiased = pd.DataFrame()
hourly_corrected = pd.DataFrame()
daily_corrected = pd.DataFrame()
global_corrected = pd.DataFrame()

features_name = ['maximum', 'minimum', 'ave', 'ave_normalized', 
            'start', 'start_normalized', 'end', 'end_normalized', 'start2end',
            'start2end_normalized', 'start2max', 'start2max_normalized', 'start2min', 'start2min_normalized', 
            'std', 'no_price', 'length', 
            'up_num', 'down_num', 'up_rate', 'down_rate', 'duration', 
            'bias_max', 'bias_min', 'bias_ave', 'bias_st']

game_name = 'iceball'
results = pd.read_csv(result_folder_path+games[game_name]+'_results.csv').set_index('Unnamed: 0')
index_list = [i[:-4] for i in os.listdir(summary_folder_path + games[game_name])]
for index in index_list:
    series_df = pd.read_csv(summary_folder_path + games[game_name]+'/'+str(index)+'.csv')
#     if len(series_df) < 7:
#         continue
    # sec 3
    sec3 = series_df[series_df['duration']<= int(series_df.iloc[-1,0]/3)] # sec 3
    # sec 2
    sec2 = series_df[(series_df['duration']<= int(series_df.iloc[-1,0]/3*2)) \
              & (series_df['duration']> int(series_df.iloc[-1,0]/3)) ] #sec 2
    # sec 1
    sec1 = series_df[series_df['duration']> int(series_df.iloc[-1,0]/3*2)] # sec 1        

    index = int(index)
    if results.loc[index,'home_score'] >  results.loc[index,'away_score'] :
        win = {'home':1,'away':0}
    else:
        win = {'home':0,'away':1}

#         these four frame will be digger by different series
    series_name = 'home'
    features = diggers(series_df,series_name)
    feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
    features = diggers(sec1,series_name)
    feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
    features = diggers(sec2,series_name)
    feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
    features = diggers(sec3,series_name)
    feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))

    features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
    unbiased = unbiased.append(features, ignore_index=True)



    series_name = 'hourly_corrected'
    features = diggers(series_df,series_name)
    feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
    features = diggers(sec1,series_name)
    feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
    features = diggers(sec2,series_name)
    feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
    features = diggers(sec3,series_name)
    feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))

    features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
    hourly_corrected = hourly_corrected.append(features, ignore_index=True)


    series_name = 'daily_corrected'
    features = diggers(series_df,series_name)
    feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
    features = diggers(sec1,series_name)
    feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
    features = diggers(sec2,series_name)
    feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
    features = diggers(sec3,series_name)
    feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))

    features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
    daily_corrected = daily_corrected.append(features, ignore_index=True)


    series_name = 'global_corrected'
    features = diggers(series_df,series_name)
    feature_dic_all = dict(zip([i+'_all' for i in features_name],features))
    features = diggers(sec1,series_name)
    feature_dic_1 = dict(zip([i+'_1' for i in features_name],features))
    features = diggers(sec2,series_name)
    feature_dic_2 = dict(zip([i+'_2' for i in features_name],features))
    features = diggers(sec3,series_name)
    feature_dic_3 = dict(zip([i+'_3' for i in features_name],features))

    features = {**feature_dic_all,**feature_dic_1,**feature_dic_2,**feature_dic_3,**win}
    global_corrected = global_corrected.append(features, ignore_index=True)

unbiased.to_csv(mining_folder_path+'/iceball_unbiased.csv')
hourly_corrected.to_csv(mining_folder_path+'/iceball_hourly_corrected.csv')
daily_corrected.to_csv(mining_folder_path+'/iceball_daily_corrected.csv')
global_corrected.to_csv(mining_folder_path+'/iceball_global_corrected.csv')